# Benchmarking

Plots the benchmarking experiment for algorithms computing the likelihood of exclusive relationships.

## Table of Contents

- [Preprocessing](#preprocessing)
- [Transforms](#transforms)
  - [Runtimes by Population Size](#runtimes-by-population-size)
  - [Runtimes by Number of Samples](#runtimes-by-number-of-samples)
  - [Runtimes by Sample Sum Percent of Population](#runtimes-by-sample-sum-percent-of-population)
- [Plots](#plots)


In [1]:
import enum
import io
import json
import lzma
import os
import zipfile
from typing import List

import chart_studio
import chart_studio.plotly as cspy
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as staticmethod

from plotly.subplots import make_subplots
from scipy import signal
from sklearn import linear_model
from sklearn.preprocessing import QuantileTransformer

## Preprocessing

In [2]:
class OptimizationType(enum.Enum):
    likelihood_of_an_exclusive_relationship_not_optimized = 'None'
    likelihood_of_an_exclusive_relationship_algebraically_optimized = 'Formula'
    likelihood_of_an_exclusive_relationship_completely_optimized = 'Formula & Algorithm'

ALGORITHM_FUNCTION_TO_LABEL_RENAME = dict(
    likelihood_of_an_exclusive_relationship_not_optimized="Not Optimized",
    likelihood_of_an_exclusive_relationship_algebraically_optimized="Function Optimized",
    likelihood_of_an_exclusive_relationship_completely_optimized="Function & Algorithm Optimized"
)
MILLISECONDS_PER_SECOND = 1000

def trial_to_records(trial: dict) -> List[dict]:
    records = []
    for result in trial['results']:
        record = dict(
            population=trial['args']['population'],
            sequence=trial['args']['sequence'],
            iterations=trial['iterations'],
            trial_id=trial['index']
        )
        record.update(result)
        records.append(record)
    return records


def calculate_percentages(df: pd.DataFrame) -> pd.DataFrame:
    df['Sample Sum Percent of Population'] = df['Sample Sum'] / df['N'] * 100
    df['Sample Mean Percent of Population'] = df['Sample Mean'] / df['N'] * 100
    df['Sample Maximum Percent of Population'] = df['Sample Maximum'] / df['N'] * 100
    df['Sample Minimum Percent of Population'] = df['Sample Minimum'] / df['N'] * 100
    return df


def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    tdf = pd.DataFrame(index=df.index)
    # Create core index
    tdf['N'] = df['population']
    tdf['Trial ID'] = df['trial_id']
    tdf['Optimization'] = df['algorithm'].replace(OptimizationType._member_map_)
    tdf['Optimization'] = tdf['Optimization'].apply(lambda _type: _type.value)


    tdf['n'] = df['sequence'].apply(len)
    tdf['A'] = df['sequence'].apply(tuple)
    tdf['Result'] = df['result']

    # Original runtime is total over the iterations
    # Average for per iteration and convert to milliseconds
    tdf['Runtime (ms)'] = df['runtime'] / df['iterations'] * MILLISECONDS_PER_SECOND

    tdf['Sample Minimum'] = tdf['A'].apply(np.min)
    tdf['Sample Maximum'] = tdf['A'].apply(np.max)
    tdf['Sample Mean'] = tdf['A'].apply(np.mean)
    tdf['Sample Sum'] = tdf['A'].apply(np.sum)

    return tdf


def pivot(df: pd.DataFrame) -> pd.DataFrame:
    pdf = df.pivot_table(
            index=['N', 'Trial ID'],
            columns=['Optimization'],
            values=['Runtime (ms)'])
    pdf = pdf.droplevel(0, axis=1)\
            .reset_index()\
            .rename_axis('', axis=1)

    gdf = df.groupby(['N', 'Trial ID'])\
            .first()\
            .drop(columns=['Optimization', 'Runtime (ms)'])
    return gdf.merge(pdf, how='left', left_index=True, right_on=['N', 'Trial ID'])


def load_benchmarking_results(path: str):
    with open(path, 'rb') as fo:
        if path.endswith('.zip'):
            zfo = zipfile.ZipFile(fo, mode='r')
            _bytes = zfo.read(zfo.filelist[0])
        elif path.endswith('.xz'):
            _bytes = lzma.decompress(fo.read(), format=lzma.FORMAT_XZ)
        else:
            _bytes: bytes = fo.read()

        data = json.loads(_bytes)

    records = []
    for trial in data['results']:
        records.extend(trial_to_records(trial))
    df = pd.DataFrame(records)

    df = preprocess(df)
    return df\
            .sort_values(['N', 'Trial ID', 'Optimization'])\
            .reset_index(drop=True)

_df = load_benchmarking_results("Results/2022-12-30 20-13.json.xz")
_pdf = pivot(_df)

_df = calculate_percentages(_df)
_pdf = calculate_percentages(_pdf)
_df

,N,Trial ID,Optimization,n,A,Result,Runtime (ms),Sample Minimum,Sample Maximum,Sample Mean,Sample Sum,Sample Sum Percent of Population,Sample Mean Percent of Population,Sample Maximum Percent of Population,Sample Minimum Percent of Population
0,10,1,Formula,3,"(2, 1, 1)",0.44,0.004907,1,2,1.333333,4,40.00,13.333333,20.00,10.00
1,10,1,Formula & Algorithm,3,"(2, 1, 1)",0.44,0.005475,1,2,1.333333,4,40.00,13.333333,20.00,10.00
2,10,1,None,3,"(2, 1, 1)",0.44,0.003097,1,2,1.333333,4,40.00,13.333333,20.00,10.00
3,10,2,Formula,3,"(1, 1, 1)",0.28,0.003237,1,1,1.000000,3,30.00,10.000000,10.00,10.00
4,10,2,Formula & Algorithm,3,"(1, 1, 1)",0.28,0.003638,1,1,1.000000,3,30.00,10.000000,10.00,10.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373645,2500,49,Formula & Algorithm,3,"(463, 650, 868)",1.00,1.410018,463,868,660.333333,1981,79.24,26.413333,34.72,18.52
373646,2500,49,None,3,"(463, 650, 868)",1.00,3.391458,463,868,660.333333,1981,79.24,26.413333,34.72,18.52
373647,2500,50,Formula,2,"(571, 1180)",1.00,3.254100,571,1180,875.500000,1751,70.04,35.020000,47.20,22.84
373648,2500,50,Formula & Algorithm,2,"(571, 1180)",1.00,1.084564,571,1180,875.500000,1751,70.04,35.020000,47.20,22.84


In [3]:
_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373650 entries, 0 to 373649
Data columns (total 15 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   N                                     373650 non-null  int64  
 1   Trial ID                              373650 non-null  int64  
 2   Optimization                          373650 non-null  object 
 3   n                                     373650 non-null  int64  
 4   A                                     373650 non-null  object 
 5   Result                                373650 non-null  float64
 6   Runtime (ms)                          373650 non-null  float64
 7   Sample Minimum                        373650 non-null  int64  
 8   Sample Maximum                        373650 non-null  int64  
 9   Sample Mean                           373650 non-null  float64
 10  Sample Sum                            373650 non-null  int64  
 11  

In [4]:
_pdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124550 entries, 0 to 124549
Data columns (total 16 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   n                                     124550 non-null  int64  
 1   A                                     124550 non-null  object 
 2   Result                                124550 non-null  float64
 3   Sample Minimum                        124550 non-null  int64  
 4   Sample Maximum                        124550 non-null  int64  
 5   Sample Mean                           124550 non-null  float64
 6   Sample Sum                            124550 non-null  int64  
 7   N                                     124550 non-null  int64  
 8   Trial ID                              124550 non-null  int64  
 9   Formula                               124550 non-null  float64
 10  Formula & Algorithm                   124550 non-null  float64
 11  

## Transforms

### Runtimes by Population Size

Groups trial results by poplation size and calculates the mean runtime for each optimization type. Applies a Savitzky-Golay filter (window=15, polynomial=3) to smooth data for plotting.

In [5]:
def population_runtime_plot_df():
        window_length = 15
        poly_order = 3
        gdf = _pdf.groupby(['N'], as_index=False).mean(numeric_only=True)[['N', 'None', 'Formula', 'Formula & Algorithm']]
        gdf['None'] = signal.savgol_filter(gdf['None'], window_length, poly_order)
        gdf['Formula'] = signal.savgol_filter(gdf['Formula'], window_length, poly_order)
        gdf['Formula & Algorithm'] = signal.savgol_filter(gdf['Formula & Algorithm'], window_length, poly_order)
        plot_df = gdf\
        .melt(
                id_vars=['N'],
                value_vars=['None', 'Formula', 'Formula & Algorithm'],
                value_name='Runtime (ms)',
                var_name='Optimization'
        )\
        .rename(columns={'N': 'Population Size'})
        # fig = px.line(
        #         plot_df,
        #         x='Population Size',
        #         y='Runtime (ms)',
        #         color='Optimization',
        #         template='plotly_dark',
        #         title="Method Performance")

        return plot_df

df_pop = population_runtime_plot_df()
df_pop

,Population Size,Optimization,Runtime (ms)
0,10,None,0.002435
1,11,None,0.003150
2,12,None,0.003736
3,13,None,0.004211
4,14,None,0.004590
...,...,...,...
7468,2496,Formula & Algorithm,1.039107
7469,2497,Formula & Algorithm,1.036876
7470,2498,Formula & Algorithm,1.038543
7471,2499,Formula & Algorithm,1.045037


### Runtimes by Number of Samples

Groups trial results by the number of samples and calculates the mean runtime for each optimization type.

In [6]:
def number_of_samples_runtime_plot_df():
        gdf = _pdf.groupby(['n'], as_index=False).mean(numeric_only=True)[['n', 'None', 'Formula', 'Formula & Algorithm']]

        plot_df = gdf\
        .melt(
                id_vars=['n'],
                value_vars=['None', 'Formula', 'Formula & Algorithm'],
                value_name='Runtime (ms)',
                var_name='Optimization'
        )\
        .rename(columns={'n': 'Number of Elements'})
        return plot_df

df_nos = number_of_samples_runtime_plot_df()
df_nos

,Number of Elements,Optimization,Runtime (ms)
0,2,None,0.992425
1,3,None,0.952171
2,4,None,0.829504
3,5,None,0.713881
4,6,None,0.611189
...,...,...,...
103,33,Formula & Algorithm,0.399429
104,34,Formula & Algorithm,0.435904
105,35,Formula & Algorithm,0.074596
106,37,Formula & Algorithm,0.276571


## Runtimes by Sample Sum Percent of Population

Rounds the percent of population sampled to the ones and groups trial results, calculating the mean runtime for each optimization type.

In [7]:
def sample_sum_percent_runtime_plot_df():
        pdf = _pdf.copy()
        pdf['Sample Sum Percent of Population'] = pdf['Sample Sum Percent of Population'].round(decimals=0)
        gdf = pdf.groupby(['Sample Sum Percent of Population'], as_index=False).mean(numeric_only=True)[['Sample Sum Percent of Population', 'None', 'Formula', 'Formula & Algorithm']]

        plot_df = gdf\
        .melt(
                id_vars=['Sample Sum Percent of Population'],
                value_vars=['None', 'Formula', 'Formula & Algorithm'],
                value_name='Runtime (ms)',
                var_name='Optimization'
        ).rename(
                columns={'Sample Sum Percent of Population': 'Percent of Population Sampled'}
        )
        return plot_df

df_ssp = sample_sum_percent_runtime_plot_df()
df_ssp

,Percent of Population Sampled,Optimization,Runtime (ms)
0,1.0,None,0.016416
1,2.0,None,0.019300
2,3.0,None,0.026122
3,4.0,None,0.036617
4,5.0,None,0.043904
...,...,...,...
295,96.0,Formula & Algorithm,0.707041
296,97.0,Formula & Algorithm,0.687166
297,98.0,Formula & Algorithm,0.710518
298,99.0,Formula & Algorithm,0.750451


## Runtimes by Likelihood of an Exclusive Relationship

Rounds the likelihood of an exclusive relationship to the thousandths place and groups trial results, calculating the mean runtime for each optimization type.

In [8]:
def result_runtime_plot_df():
        pdf = _pdf.copy()
        pdf['Result'] = pdf['Result'].round(decimals=3)
        gdf = pdf.groupby(['Result'], as_index=False).mean(numeric_only=True)[['Result', 'None', 'Formula', 'Formula & Algorithm']]

        plot_df = gdf\
        .melt(
                id_vars=['Result'],
                value_vars=['None', 'Formula', 'Formula & Algorithm'],
                value_name='Runtime (ms)',
                var_name='Optimization'
        ).rename(
                columns={'Result': 'Likelihood of an Exclusive Relationship'}
        )
        return plot_df

df_r = result_runtime_plot_df()
df_r

,Likelihood of an Exclusive Relationship,Optimization,Runtime (ms)
0,0.011,None,0.003532
1,0.012,None,0.015057
2,0.013,None,0.003313
3,0.015,None,0.002597
4,0.016,None,0.005755
...,...,...,...
2812,0.996,Formula & Algorithm,0.052183
2813,0.997,Formula & Algorithm,0.035820
2814,0.998,Formula & Algorithm,0.048823
2815,0.999,Formula & Algorithm,0.051543


## Plots

In [9]:
fig = make_subplots(
    rows=2,
    cols=3,
    specs=[[{"secondary_y": True}, {"secondary_y": True}, {"secondary_y": True}],
           [{"colspan": 3}, None, None]],
    shared_yaxes=True)
fig.update_layout(title_text="Did the optimization strategy improve performance?", legend_title="What is optimized?")
# Primary y
fig.update_yaxes(title_text="Runtime (ms)", row=1, col=1, secondary_y=False)
fig.update_yaxes(title_text="Runtime (ms)", row=2, col=1)
# Secondary y
fig.update_yaxes(title=dict(text="Runtime Reduction (%)", font=dict(size=12)), row=1, col=3, secondary_y=True)

OPTIMIZATIONS = ['None', 'Formula', 'Formula & Algorithm']
LEGEND_RENAME_MAP = {'None': 'Unoptimized'}
LEGEND_NAME_TRACE_COLOR_MAP = {
    'Unoptimized': plotly.colors.colorbrewer.Dark2_r[0],
    'Formula': plotly.colors.colorbrewer.Dark2_r[1],
    'Formula & Algorithm': plotly.colors.colorbrewer.Dark2_r[2]
}

def add_scatter_traces_for_optimizations(df: pd.DataFrame, independent: str, row: int, col: int, title_text: str, fig: go.Figure, add_percent_improvement: bool = True) -> go.Figure:
    # Subplot title will be on X axis
    fig.update_xaxes(title_text=title_text, row=row, col=col)
    for i, optimization in enumerate(OPTIMIZATIONS):
        name = LEGEND_RENAME_MAP.get(optimization, optimization)
        color = LEGEND_NAME_TRACE_COLOR_MAP[name]

        # Create a trace on the subplot for the given optimization
        # Include the trace in the subplot's legend group
        trace = go.Scatter(
                x=df[df['Optimization'] == optimization][independent],
                y=df[df['Optimization'] == optimization]['Runtime (ms)'],
                name=name,
                marker=go.scatter.Marker(color=color),
                legendgroup=title_text
        )
        trace.legendgrouptitle.text = title_text
        fig.add_trace(trace, row=row, col=col, secondary_y=False)

    if add_percent_improvement:
        # Include a trace on the plot that shows the percent reduction in runtime from
        # the unoptimized formula to the optimized formula & algorithm

        piv = df.pivot_table(values="Runtime (ms)", index=[independent], columns="Optimization")

        # Smooth the percent change with the Savitzky-Golay filter
        #  - create a window 10% of the total number of rows
        #  - polynomial of 3
        window_length = int(piv.shape[0] * 0.1)
        if window_length % 2 == 0:
            window_length += 1
        if window_length <= 3:
            window_length = 5
        piv['Runtime Reduction'] = signal.savgol_filter(
                (1 - piv['Formula & Algorithm'] / piv['None']) * 100,
                window_length=window_length,
                polyorder=3)

        trace = go.Scatter(
                x=piv.index,
                y=piv['Runtime Reduction'],
                name="% Reduction",
                showlegend=False,
                marker=go.scatter.Marker(
                    color=plotly.colors.colorbrewer.Dark2_r[4]),
                line=dict(dash='dash'),
                legendgroup=title_text,
                opacity=0.5
        )
        fig.add_trace(trace, row=row, col=col, secondary_y=True)

    # Percent Gain
    return fig


fig = add_scatter_traces_for_optimizations(df_pop, 'Population Size', 1, 1, 'Population Size', fig)
fig = add_scatter_traces_for_optimizations(df_nos, 'Number of Elements', 1, 2, 'Number of Samples', fig)
fig = add_scatter_traces_for_optimizations(df_ssp, 'Percent of Population Sampled', 1, 3, ' Population Sampled (%)', fig)
fig = add_scatter_traces_for_optimizations(df_r, 'Likelihood of an Exclusive Relationship', 2, 1, 'Likelihood of an Exclusive Relationship', fig, False)


annotations = []
# Add compute environment & system information
annotations.append(dict(xref="paper", yref="paper",
                              x=.5, y=.45,
                              xanchor='center', yanchor='middle',
                              text='Macbook Pro 2016, macOS Monterey 12.5.1, Python 3.10.4, 2.9GHz Dual-Core Intel Core i5',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))

fig.update_layout(template='plotly_dark', legend=dict(groupclick="toggleitem"), annotations=annotations)
# Legend neither displays properly nor uses 'toggleitem' on Plotly Chart Studio online
fig.update_layout(showlegend=False)

fig

In [10]:
# Upload to Plotly Chart Studio online
username: str = ''
api_key: str = ''
# chart_studio.tools.set_credentials_file(username, api_key)
# cspy.plot(fig, filename = fig.layout.title.text, auto_open=True)